# (baseline development) Silicon per M2 Calculations

This journal documents the methods and assumptions made to create a baseline material file for silicon.

## Mass per M2

The mass of silicon contained in a PV module is dependent on the size, thickness and number of cells in an average module. Since there is a range of sizes and number of cells per module, we will attempt a weighted average. These weighted averages are based on ITRPV data, which goes back to 2010, Fraunhofer data back to 1990, and 

In [32]:
import numpy as np
import pandas as pd
import os,sys

density_si = 2.3290 #g/cm^3 from Wikipedia of Silicon (https://en.wikipedia.org/wiki/Silicon) 
#it might be better to have mono-Si and multi-Si densities, including dopants, 
#but that density is not readily available

A Fraunhofer report indicates that in 1990, wafers were 400 micron thick, decreasing to the more modern 180 micron thickness by 2008. ITRPVs back to 2010 indicate that 156 mm x 156mm was the standard size wafer through 2015.

In [33]:
#weighted average for wafer size 2016 through 2030
#2016
wafer2016mcsi = (0.90*156 + 0.10*156.75)/100
wafer2016monosi = (0.55*156 + 0.45*156.75)/100
wafer2016avg = 0.9*wafer2016mcsi + 0.1*wafer2016monosi
print("Average Wafer size in 2016 was", wafer2016avg, "cm on a side")

Average Wafer size in 2016 was 1.5610125000000001 cm on a side


In [43]:
#now lets try to do this for 2019 through 2030 all at once with dataframes
#taking the average of the ranges specified in ITRPVs

#first we input the market share data for mcSi and monoSi, read in from csv
cwd = os.getcwd() #grabs current working directory
mrktshr_cellsize = pd.read_csv(cwd+"/../../CEMFC/baselines/SupportingMaterial/MarketShare_CellSize.csv", index_col='Year')
mrktshr_cellsize /=100 #turn whole numbers into decimal percentages
#print(mrktshr_cellsize)

#then split them into two dataframes for later computations
dfmarketshare_mcSi = mrktshr_cellsize.filter(regex = 'mcSi')
dfmarketshare_monoSi = mrktshr_cellsize.filter(regex = 'monoSi')
#adjust column names for matching computation later
dfmarketshare_mcSi.columns = ['share156','share156.75','share157.75','share163','share166up']
dfmarketshare_monoSi.columns = ['share156','share156.75','share157.75','share163','share166up']

print(dfmarketshare_mcSi)
print(dfmarketshare_monoSi)


      share156  share156.75  share157.75  share163  share166up
Year                                                          
1995      1.00          NaN          NaN       NaN         NaN
1996      1.00          NaN          NaN       NaN         NaN
1997      1.00          NaN          NaN       NaN         NaN
1998      1.00          NaN          NaN       NaN         NaN
1999      1.00          NaN          NaN       NaN         NaN
2000      1.00          NaN          NaN       NaN         NaN
2001      1.00          NaN          NaN       NaN         NaN
2002      1.00          NaN          NaN       NaN         NaN
2003      1.00          NaN          NaN       NaN         NaN
2004      1.00          NaN          NaN       NaN         NaN
2005      1.00          NaN          NaN       NaN         NaN
2006      1.00          NaN          NaN       NaN         NaN
2007      1.00          NaN          NaN       NaN         NaN
2008      1.00          NaN          NaN       NaN     

Interpolate marketshare for missing years in ITRPV 2020 predictions
----
choosing to interpolate market share of different sizes rather than cell size because this should be more basedin technology - i.e. crystals only grow certain sizes. Additionally, it is more helpful to understand the impact silicon usage by keeping cell size and marketshare seperate.

In [57]:
#interpolate for missing marketshare data
##the interpolate function returns a view of the df, doesn't modify the df itself
##therefore you have to set the old df, or a new one = the df.interpolate function
dfmarketshare_mcSi=dfmarketshare_mcSi.interpolate(method='linear',axis=0,limit=2,limit_area='inside')
dfmarketshare_monoSi=dfmarketshare_monoSi.interpolate(method='linear',axis=0,limit=2,limit_area='inside')

#fill remaining NaN/outside with 0 (i.e., no market share)
dfmarketshare_mcSi=dfmarketshare_mcSi.fillna(0.0)
dfmarketshare_monoSi=dfmarketshare_monoSi.fillna(0.0)

print(dfmarketshare_mcSi)
print(dfmarketshare_monoSi)

      share156  share156.75  share157.75  share163  share166up
Year                                                          
1995      1.00     0.000000     0.000000  0.000000    0.000000
1996      1.00     0.000000     0.000000  0.000000    0.000000
1997      1.00     0.000000     0.000000  0.000000    0.000000
1998      1.00     0.000000     0.000000  0.000000    0.000000
1999      1.00     0.000000     0.000000  0.000000    0.000000
2000      1.00     0.000000     0.000000  0.000000    0.000000
2001      1.00     0.000000     0.000000  0.000000    0.000000
2002      1.00     0.000000     0.000000  0.000000    0.000000
2003      1.00     0.000000     0.000000  0.000000    0.000000
2004      1.00     0.000000     0.000000  0.000000    0.000000
2005      1.00     0.000000     0.000000  0.000000    0.000000
2006      1.00     0.000000     0.000000  0.000000    0.000000
2007      1.00     0.000000     0.000000  0.000000    0.000000
2008      1.00     0.000000     0.000000  0.000000    0

In [58]:
#multiply each marketshare dataframe column by it's respective size
#dfmarketshare_mcSi.share156 *=156 #this is a slow way to multiply each column by its respective size

cellsizes = {'share156':156,
            'share156.75':156.75,
            'share157.75':157.75,
            'share163':163.875,
            'share166up':166} #dictionary of the average cell dimension for each market share bin (ITRPV 2020)

#multiply cell dimensions by their market share to get a weighted average
##this is where the column names needed to match
df_scalecell_mcSi = dfmarketshare_mcSi.mul(cellsizes,'columns')
df_scalecell_monoSi = dfmarketshare_monoSi.mul(cellsizes,'columns')

print(df_scalecell_mcSi)
print(df_scalecell_monoSi)

      share156  share156.75  share157.75  share163  share166up
Year                                                          
1995    156.00      0.00000     0.000000    0.0000    0.000000
1996    156.00      0.00000     0.000000    0.0000    0.000000
1997    156.00      0.00000     0.000000    0.0000    0.000000
1998    156.00      0.00000     0.000000    0.0000    0.000000
1999    156.00      0.00000     0.000000    0.0000    0.000000
2000    156.00      0.00000     0.000000    0.0000    0.000000
2001    156.00      0.00000     0.000000    0.0000    0.000000
2002    156.00      0.00000     0.000000    0.0000    0.000000
2003    156.00      0.00000     0.000000    0.0000    0.000000
2004    156.00      0.00000     0.000000    0.0000    0.000000
2005    156.00      0.00000     0.000000    0.0000    0.000000
2006    156.00      0.00000     0.000000    0.0000    0.000000
2007    156.00      0.00000     0.000000    0.0000    0.000000
2008    156.00      0.00000     0.000000    0.0000    0

In [59]:
#now add the columns together to get the weighted average cell size for each year for each technology
df_avgcell_mcSi = pd.DataFrame(df_scalecell_mcSi.agg("sum", axis="columns"))
df_avgcell_monoSi = pd.DataFrame(df_scalecell_monoSi.agg("sum", axis="columns")) #agg functions return a series not a dictionary
#print(df_avgcell_mcSi)

#join the two dataframes into single one with two columns
df_avgcell = pd.concat([df_avgcell_monoSi,df_avgcell_mcSi], axis=1) #concatinate on the columns axis
df_avgcell.columns = ['monoSi','mcSi'] #name the columns
print(df_avgcell)


          monoSi        mcSi
Year                        
1995  156.000000  156.000000
1996  156.000000  156.000000
1997  156.000000  156.000000
1998  156.000000  156.000000
1999  156.000000  156.000000
2000  156.000000  156.000000
2001  156.000000  156.000000
2002  156.000000  156.000000
2003  156.000000  156.000000
2004  156.000000  156.000000
2005  156.000000  156.000000
2006  156.000000  156.000000
2007  156.000000  156.000000
2008  156.000000  156.000000
2009  156.000000  156.000000
2010  156.000000  156.000000
2011  156.000000  156.000000
2012  156.000000  156.000000
2013  156.000000  156.000000
2014  156.000000  156.000000
2015  156.000000  156.000000
2016  156.337500  156.075000
2017  156.663750  156.472500
2018  174.973750  156.675000
2019  157.663750  157.460000
2020  159.990000  159.912500
2021  160.872500  161.008750
2022  161.755000  162.105000
2023  163.099375  154.655000
2024  164.443750  147.205000
2025  164.768333  153.116667
2026  165.092917  159.028333
2027  165.4175

Now we have an average cell dimension for mc-Si and mono-Si for 2016 through 2030. Next, we apply the marketshare of mc-Si vs mono-Si to get the average cell dimension for the year. Market share of mc-Si vs mono-Si is taken from LBNL "Tracking the Sun" report (warning: this is non-utility scale data i.e. <5MW, and is from 2002-2018), from Mints 2019 SPV report, from ITRPVs, and old papers (Costello & Rappaport 1980, Maycock 2003 & 2005).

In [60]:
#read in a csv that was copied from CE Data google sheet
cwd = os.getcwd() #grabs current working directory
techmarketshare = pd.read_csv(cwd+"/../../CEMFC/baselines/SupportingMaterial/ModuleType_MarketShare.csv",index_col='Year')
#this file path navigates from current working directory back up 2 folders, and over to the csv
techmarketshare /=100 #turn whole numbers into decimal percentages
techmarketshare['Year'] = techmarketshare.index #add the year back in for later merge
print(techmarketshare)

      monoSi_LBNL  mcSi_LBNL  otherSi_LBNL  monoSi_Mints  mcSi_Mints  \
Year                                                                   
1980          NaN        NaN           NaN           NaN         NaN   
1981          NaN        NaN           NaN           NaN         NaN   
1982          NaN        NaN           NaN           NaN         NaN   
1983          NaN        NaN           NaN           NaN         NaN   
1984          NaN        NaN           NaN           NaN         NaN   
1985          NaN        NaN           NaN           NaN         NaN   
1986          NaN        NaN           NaN           NaN         NaN   
1987          NaN        NaN           NaN           NaN         NaN   
1988          NaN        NaN           NaN           NaN         NaN   
1989          NaN        NaN           NaN           NaN         NaN   
1990          NaN        NaN           NaN           NaN         NaN   
1991          NaN        NaN           NaN           NaN        

#### create a harmonization of annual market share, and interpolate

In [62]:
# first, create a single value of tech market share in each year or NaN
#split mcSi and monoSi
mcSi_cols = techmarketshare.filter(regex = 'mcSi')
monoSi_cols = techmarketshare.filter(regex = 'mono')
#print (mcSi_cols)

#aggregate all the columns of mono or mcSi into one averaged market share
est_mktshr_mcSi = pd.DataFrame(mcSi_cols.agg("mean", axis="columns"))
#print(est_mktshr_mcSi)
est_mktshr_monoSi = pd.DataFrame(monoSi_cols.agg("mean", axis="columns"))
#print(est_mktshr_monoSi)

#Join the monoSi and mcSi back together as a dataframe
est_mrktshrs = pd.concat([est_mktshr_monoSi,est_mktshr_mcSi], axis=1) #concatinate on the columns axis
est_mrktshrs.columns = ['monoSi','mcSi'] #name the columns


#sanity check of market share data - does it add up?
est_mrktshrs['Total'] = est_mrktshrs.monoSi+est_mrktshrs.mcSi

print(est_mrktshrs)
#Warning: 2002, 10% of the silicon marketshare was "other", including amorphous, etc.
del est_mrktshrs['Total']

#interpolate marketshares for each year

        monoSi      mcSi     Total
Year                              
1980  1.000000       NaN       NaN
1981       NaN       NaN       NaN
1982       NaN       NaN       NaN
1983       NaN       NaN       NaN
1984       NaN       NaN       NaN
1985       NaN       NaN       NaN
1986       NaN       NaN       NaN
1987       NaN       NaN       NaN
1988       NaN       NaN       NaN
1989       NaN       NaN       NaN
1990       NaN       NaN       NaN
1991       NaN       NaN       NaN
1992       NaN       NaN       NaN
1993  0.620000  0.380000  1.000000
1994       NaN       NaN       NaN
1995       NaN       NaN       NaN
1996       NaN       NaN       NaN
1997       NaN       NaN       NaN
1998       NaN       NaN       NaN
1999  0.466000  0.534000  1.000000
2000  0.389000  0.611000  1.000000
2001  0.425000  0.575000  1.000000
2002  0.220000  0.680000  0.900000
2003  0.080000  0.900000  0.980000
2004  0.080000  0.920000  1.000000
2005  0.309000  0.691000  1.000000
2006  0.270000  0.73

In [66]:
#Interpolate for marketshare NaN values
est_mrktshrs['mcSi'][1980]=0.0
est_mrktshrs = est_mrktshrs.interpolate(method='linear',axis=0,limit_area='inside')

#sanity check of market share data - does it add up?
est_mrktshrs['Total'] = est_mrktshrs.monoSi+est_mrktshrs.mcSi
print(est_mrktshrs)
#Warning: 2002, 10% of the silicon marketshare was "other", including amorphous, etc.
del est_mrktshrs['Total']

        monoSi      mcSi     Total
Year                              
1980  1.000000  0.000000  1.000000
1981  0.970769  0.029231  1.000000
1982  0.941538  0.058462  1.000000
1983  0.912308  0.087692  1.000000
1984  0.883077  0.116923  1.000000
1985  0.853846  0.146154  1.000000
1986  0.824615  0.175385  1.000000
1987  0.795385  0.204615  1.000000
1988  0.766154  0.233846  1.000000
1989  0.736923  0.263077  1.000000
1990  0.707692  0.292308  1.000000
1991  0.678462  0.321538  1.000000
1992  0.649231  0.350769  1.000000
1993  0.620000  0.380000  1.000000
1994  0.594333  0.405667  1.000000
1995  0.568667  0.431333  1.000000
1996  0.543000  0.457000  1.000000
1997  0.517333  0.482667  1.000000
1998  0.491667  0.508333  1.000000
1999  0.466000  0.534000  1.000000
2000  0.389000  0.611000  1.000000
2001  0.425000  0.575000  1.000000
2002  0.220000  0.680000  0.900000
2003  0.080000  0.900000  0.980000
2004  0.080000  0.920000  1.000000
2005  0.309000  0.691000  1.000000
2006  0.270000  0.73

In [146]:
#now combine technology market share of mcSi and monoSi with their respective cell dimensions
#which have already been cell size marketshare weighted
#going to ignore "otherSi" because for the most part less than 2%, except 2002

#join the mcSi together and the monoSi together in separate dataframes
monoSicell = pd.DataFrame(df_avgcell.monoSi)
monoSimarket = pd.DataFrame(est_mrktshrs.monoSi)

df_monoSi = monoSimarket.join(monoSicell, how='left',lsuffix='_share', rsuffix='_cell') 
print(df_monoSi)

      monoSi_share  monoSi_cell
Year                           
1980      1.000000          NaN
1981           NaN          NaN
1982           NaN          NaN
1983           NaN          NaN
1984           NaN          NaN
1985           NaN          NaN
1986           NaN          NaN
1987           NaN          NaN
1988           NaN          NaN
1989           NaN          NaN
1990           NaN          NaN
1991           NaN          NaN
1992           NaN          NaN
1993      0.620000          NaN
1994           NaN          NaN
1995           NaN          NaN
1996           NaN          NaN
1997           NaN          NaN
1998           NaN          NaN
1999      0.466000          NaN
2000      0.389000          NaN
2001      0.425000          NaN
2002      0.220000          NaN
2003      0.080000          NaN
2004      0.080000          NaN
2005      0.309000          NaN
2006      0.270000          NaN
2007      0.350000          NaN
2008      0.450000          NaN
2009    